In [7]:
import cbpro
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import yaml
import time

yaml_filename = 'jay_cbpro_config.yaml'
my_yaml_file = open(f"../../{yaml_filename}")

yaml_dict = yaml.load(my_yaml_file, Loader=yaml.FullLoader)

locals().update(yaml_dict["cbpro"])

auth_client = cbpro.AuthenticatedClient(API_KEY, API_SECRET, API_PASS)
pub_client = cbpro.PublicClient()

# Pseudocode for live trading bot

// API call every 5 minute

IF the time is 1 day candle:
    
    THEN clear all positions (market sell all existing orders)
    
ELSE:

    get a list of top 4 coins sorted by USD volume
    
    num_orders = the number of coins with existing orders
    
    FOR coin IN list:
        IF there is an existing order:
            PASS
        ELSE:
            get buy signal based on k = 0.6
            market buy with (total cash / (4 - num_orders)

In [8]:
# get account_info for holding coins
acc_info = {}

for data in auth_client.get_accounts():
    currency = data['currency']
    acc_info[currency] = data
        
acc_info['USD']

{'id': '4cf15f9c-67cf-42c2-8fda-9190451eb472',
 'currency': 'USD',
 'balance': '1814.1770972183245000',
 'hold': '0.0000000000000000',
 'available': '1814.1770972183245',
 'profile_id': '6cf41155-d4f1-405e-82a9-bc7a7515d039',
 'trading_enabled': True}

In [6]:
for coin in acc_info.keys():
    coin_size = float(acc_info[coin]['available'])
    if coin_size > 1. and 'USD' not in coin:
        price = float(pub_client.get_product_24hr_stats(f'{coin}-USD')['last'])
        usd_size = round(price * coin_size, 2)
        print(f'Selling {coin}, size: ${usd_size}')
        print(auth_client.place_market_order(product_id=f'{coin}-USD', side='sell', funds=usd_size))
        time.sleep(0.1)

In [ ]:
import time

start_time = time.time()

cur_time = pub_client.get_product_ticker('BTC-USD')['time']
parsed_time = datetime.strptime(cur_time, '%Y-%m-%dT%H:%M:%S.%fZ')

num_ticker = 4
k = 0.6

df_dict = {}

# if the time is 1 day candle, clear all positions
if (parsed_time.minute == 0) & (parsed_time.hour == 0):
    for coin in acc_info.keys():
        currency = acc_info[coin]['currency']
        size = float(acc_info[coin]['available'])
        auth_client.place_market_order(product_id=f'{coin}-USD', side='sell', funds=size)

# if it's not 1 day candle, monitor prices
else:
    
    # get candle info 
    for coin in acc_info.keys():
        df = pd.DataFrame(pub_client.get_product_historic_rates(f'{coin}-USD', granularity=24*60*60),
                      columns=['time','low','high','open','close','volume']).sort_values('time')
        df['time'] = pd.to_datetime(df['time'],unit='s')
        df.set_index('time', inplace=True)
        df_dict[coin] = df.iloc[-2:,:] # we need today & yesterday's data
    
    # get volume in usd
    df_vol = pd.DataFrame(columns=['coin','vol_usd'])
    for coin in df_dict.keys():
        try:
            past_data = df_dict[coin].iloc[0,:]
            vol_usd = past_data['volume'] * past_data['close']
            df_vol = df_vol.append({'coin':coin, 'vol_usd':vol_usd}, ignore_index=True)
        except:
            pass

    # list top X coins by volume
    final_list = df_vol.sort_values('vol_usd', ascending=False).head(num_ticker).coin.tolist()
    print(f'Final list: {final_list}')

    # function for getting buy signal
    def get_signal(past_high, past_low, cur_open, cur_high, k):
        price_range = past_high - past_low

        # if current day's price has satisfied the standard, we buy
        if cur_high > cur_open + price_range * k:
            buy_price = cur_open + price_range * k
            return buy_price
        else:
            # no buy signal
            return -1
        
    # get list of coins we hold
    coins_holding_list = []
    for data in acc_info.values():
        coin = data['currency']
        coin_amount = float(data['available'])
        if coin_amount > 0 and 'USD' not in coin:
            price = float(pub_client.get_product_24hr_stats(f'{coin}-USD')['last'])
            if coin_amount * price > 0.1:
                coins_holding_list.append(coin)
    num_coins_holding = len(coins_holding_list)
    print(coins_holding_list)
    
    # market buy coins
    for coin in final_list:
        
        # if we don't already have a position in this coin, we buy
        if coin not in coins_holding_list:
            past_data = df_dict[coin].iloc[0,:]
            cur_data = df_dict[coin].iloc[1,:]
            cur_price = float(pub_client.get_product_24hr_stats(f'{coin}-USD')['last'])

            buy_price = get_signal(past_data['high'], past_data['low'], cur_data['open'], cur_price, k)

            # if we have buy signal
            if buy_price > 0:
                size = round(float(acc_info['USD']['available'])/(num_ticker - num_coins_holding), 2)
                auth_client.place_market_order(product_id=f'{coin}-USD', side='buy', funds=size)
                num_coins_holding += 1
                print(f'Purchased: {coin}')
                print(f'Size: ${size}')

end_time = time.time()

print(end_time-start_time)

In [ ]:
df = pd.DataFrame(pub_client.get_product_historic_rates(f'BTC-USD', granularity=24*60*60),
              columns=['time','low','high','open','close','volume']).sort_values('time')
df['time'] = pd.to_datetime(df['time'],unit='s')
df.set_index('time', inplace=True)
df_dict['BTC'] = df.iloc[-2:,:] # we need today & yesterday's data

In [ ]:
df_dict['BTC']

In [ ]:
acc_info['USD']